# FinanceHer Receipt Scanner with Google AI Studio & Python

Since Google Colab does not have direct access to your local files, you need to upload the receipt image manually or mount Google Drive.

# Receipt Scanning

In [ ]:
# Run the following code in a Colab cell to upload your receipt manually:

from google.colab import files

uploaded = files.upload()  # This prompts file selection from your device

for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")
    file_path = f"/content/{filename}"  # Define the file path for later use

Now set Up Google AI Studio API Key

In [ ]:
API_KEY = ""  # Replace this with your actual API key

Now process the Image with Google AI Studio Now, we send the receipt image to Google AI Studio's Gemini API for text extraction.

In [ ]:
!pip install requests pillow

In [ ]:
# Convert Image to Base64 (for API Request)

import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Use the uploaded file path
base64_image = encode_image(file_path)


Send Image to Google AI Studio for OCR

In [ ]:
import requests
import json

url = f"https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent?key={API_KEY}"

# If gemini-1.5-flash is overloaded, try using gemini-1.5-pro:
# url = f"https://generativelanguage.googleapis.com/v1/models/gemini-1.5-pro:generateContent?key={API_KEY}"

headers = {"Content-Type": "application/json"}

data = {
    "contents": [{
        "parts": [
            {"text": "Extract and return all text from this receipt image."},  # Add a text prompt
            {
                "inline_data": {
                    "mime_type": "image/png",  # Change to "image/png" if needed
                    "data": base64_image
                }
            }
        ]
    }]
}

response = requests.post(url, headers=headers, json=data)
result = response.json()

# Extract text from response
try:
    extracted_text = result["candidates"][0]["content"]["parts"][0]["text"]
    print("Extracted Receipt Text:\n", extracted_text)
except KeyError:
    print("Error: Unable to extract text. Check API response:", result)

# Organize Spending Into Visual Data

Now categorize expenses:

After extracting the text, you can categorize expenses using simple keyword matching:

In [ ]:
# difflib provides tools for comparing sequences, finding similarities, and determining approximate matches
import difflib  # Helps with fuzzy matching

# Predefined categories with items
categories = {
    "bananas": "Groceries",
    "wine bottle": "Groceries",
    "dinning table": "Furniture",
    "beer case": "Groceries",
    "bluetooth care": "Technology",
    "hand towel": "Household",
    "gatorade": "Groceries",
    "t-shirt": "Clothing",
    "push pins": "Office Supplies",
    "big 42 inch led tv": "Technology",
    "bluetooth": "Technology",
    "dave shanpoo": "Personal Care",
    "dave conditioner": "Personal Care"
}

def categorize_expenses(text):
    # Initialize an empty dictionary to store expenses
    expenses = {"Groceries": 0, "Furniture": 0, "Technology": 0, "Household": 0, "Clothing": 0, "Office Supplies": 0, "Personal Care": 0}

    # Split the receipt text into lines
    lines = text.lower().split("\n")  # Convert to lowercase for consistency

    for line in lines:
        # For each line, check if any category item is found
        for item, category in categories.items():
            if item in line:  # Case-insensitive match for the item in line
                expenses[category] += 1  # Increment the category count if a match is found
                break  # Stop checking once matched

    # Remove categories with 0 items (we will not include them in the final result)
    expenses = {category: count for category, count in expenses.items() if count > 0}

    return expenses

categorized_expenses = categorize_expenses(extracted_text)
print("Categorized Expenses:", categorized_expenses)

Now display expenses in a Pie Chart To visualize the categorized expenses:

In [ ]:
import matplotlib.pyplot as plt # For plotting the pie and bar chart

def display_pie_chart(expenses):
    # Only display categories with non-zero counts
    categories = list(expenses.keys())
    values = list(expenses.values())

    # Create pie chart
    plt.figure(figsize=(7, 7))
    plt.pie(values, labels=categories, autopct='%1.1f%%', startangle=140)
    plt.title('Categorized Expenses')
    plt.axis('equal')  # Equal aspect ratio ensures pie chart is circular.
    plt.show()

# Example usage: display the pie chart based on categorized expenses
display_pie_chart(categorized_expenses)

Determine the amount spent on each category and display in a horizontal bar graph

In [ ]:
import re  # Regular expression for extracting amounts

def totalize_expenses(text):
    # Initialize an empty dictionary to store expenses (by category)
    expenses = {"Groceries": 0.0, "Furniture": 0.0, "Technology": 0.0, "Household": 0.0, "Clothing": 0.0, "Office Supplies": 0.0, "Personal Care": 0.0}

    # Split the receipt text into lines
    lines = text.lower().split("\n")  # Convert to lowercase for consistency

    for line in lines:
        found = False

        # Ignore non-item-related lines (like subtotal, tax, and total)
        if any(keyword in line for keyword in ["subtotal", "tax", "total", "visa charge"]):
            continue

        # Try to extract the amount from the line using regular expressions (e.g., $3.99, 12.99)
        amount = re.search(r"\$?(\d+\.\d{2})", line)  # Optional $ symbol before amount
        if amount:
            amount = float(amount.group(1))  # Extract the amount (float)
        else:
            continue  # Skip lines where there's no valid amount

        # Check if the line contains a category item
        for item, category in categories.items():
            if item in line:
                expenses[category] += amount  # Add the extracted amount to the category's total
                found = True
                break  # Stop checking once matched

    # Remove categories with 0.0 spending (no items for that category)
    expenses = {category: spending for category, spending in expenses.items() if spending > 0.0}

    return expenses

# Categorize expenses based on the extracted text
totalized_expenses = totalize_expenses(extracted_text)
print("Categorized Expenses:", totalized_expenses)

Now display expenses in a Pie Chart To visualize the totalized expenses:

In [ ]:
def display_horizontal_bar_graph(expenses):
    # Prepare the data for the bar chart
    categories = list(expenses.keys())
    values = list(expenses.values())

    # Create horizontal bar chart
    plt.figure(figsize=(10, 6))
    bars = plt.barh(categories, values, color='skyblue')
    plt.xlabel('Amount Spent ($)')
    plt.title('Amount Spent in Each Category')
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Display the amount spent at the end of each bar
    for bar in bars:
        plt.text(bar.get_width() + 10, bar.get_y() + bar.get_height() / 2, f'${bar.get_width():.2f}', va='center', ha='left', color='black')

    plt.show()

# Display the horizontal bar graph based on categorized expenses
display_horizontal_bar_graph(totalized_expenses)

# Recieve receive advice on what to do next
(Based off your current spending habits/financial goals)

Use Google Gemini as a chatbot to give financial advice

In [ ]:
!pip install google-generativeai --quiet

In [ ]:
import google.generativeai as genai

In [ ]:
# Base URL for Gemini API (using gemini-1.5-flash to match your cURL request)
API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?key={API_KEY}"

In [ ]:
# Function to call the Gemini API
def call_gemini(prompt):
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "contents": [{
            "parts": [{"text": prompt}]
        }]
    }

    response = requests.post(API_URL, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return response.json()["candidates"][0]["content"]["parts"][0]["text"]
    else:
        return f"Error {response.status_code}: {response.text}"

In [ ]:
# Define financial advisor personality directly
personality = "You are a professional financial advisor. Provide sound financial advice in simple and uncomplicated language, including budgeting, investing, saving, and managing personal finances. Ensure the advice is clear, empathetic, and easy for anyone to understand, regardless of their financial knowledge. Additionally, keep the advice short and to the point"

# Function to interact with the AI chatbot
def chat_with_ai(user_message):
    prompt = f"You are a chatbot with the following personality: {personality}. Respond to this message:\n\n{user_message}"
    return call_gemini(prompt)

print("\nYou're chatting with a Financial Advisor 💼 AI! Type 'exit' to stop.\n")

while True:
    user_message = input("You: ")
    if user_message.lower() == "exit" or user_message.lower() == "bye" or user_message.lower() == "goodbye":
        print("Goodbye! 👋")
        break

    ai_response = chat_with_ai(user_message)
    print(f"Financial Advisor 💼: {ai_response}\n")